<a href="https://colab.research.google.com/github/InsightofSPb/Introduction_to_ML_and_Advanced_ML/blob/main/Lab12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade gym==0.18.0
import gym
import numpy as np
import time
from IPython.display import clear_output
from tqdm import tqdm
import matplotlib.pyplot as plt

epsilon = 0.1 # Параметр эпсилон при использовании эпсилон жадной стратегии
gamma = 0.8 # Коэффциент дисконтирования гамма
random_seed = 3 #Random seed
time_delay = 1 # Задержка времени при отрисовке процесса игры после обучения (секунды)
lr_rate = 0.9 #Коэффициент скорости обучения альфа

#Создание карты
def generate_random_map(size, p, sd):
    """Generates a random valid map (one that has a path from start to goal)
    :param size: size of each side of the grid
    :param p: probability that a tile is frozen
    """
    valid = False
    np.random.seed(sd)

    # DFS to check that it's a valid path.
    def is_valid(res):
        frontier, discovered = [], set()
        frontier.append((0,0))
        while frontier:
            r, c = frontier.pop()
            if not (r,c) in discovered:
                discovered.add((r,c))
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == 'G':
                        return True
                    if (res[r_new][c_new] not in '#H'):
                        frontier.append((r_new, c_new))
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(['F', 'H'], (size, size), p=[p, 1-p])
        res[0][0] = 'S'
        res[-1][-1] = 'G'
        valid = is_valid(res)
    return ["".join(x) for x in res]

#Генерация карты
random_map = generate_random_map(size=6, p=0.8, sd=random_seed) #Создаем свою карту
env = gym.make("FrozenLake-v0", desc=random_map, is_slippery=False) #Инициализируем среду
print("Ваша карта")
env.render() #Выводим карту на экран


#Задание алгоритма Q - Обучения
def choose_action(state):
    action=0
    if np.random.uniform(0, 1) < epsilon:
        action = np.random.randint(0,env.action_space.n) #***
    else:
        action = np.random.choice(np.array(np.argwhere(Q[state, :] == np.amax(Q[state, :])).flatten().tolist()))
    return action
# argwhere() ищет индекс элемента по условию, flatten() - возвращает 1D ряд, tolist() - обратная array([])
# amax() - максимальное значение вдоль строки
def learn(state, state2, reward, action, done):
    if done:
        Q[state, action] = Q[state, action] + lr_rate * (reward - Q[state, action]) # терминальное состояние
    else:
        Q[state, action] = Q[state, action] + lr_rate * (reward + gamma * np.max(Q[state2,:]) - Q[state, action])

from tqdm import tqdm
# Inititalization
np.random.seed(random_seed)
wins = []
total_games = 10000
max_steps = 100
Q = np.zeros((env.observation_space.n, env.action_space.n))
episode = 0
#Main cycle
for game in tqdm(range(total_games)):
    state = env.reset()
    t = 0
    while t < max_steps:
        if game > 5 and wins[game-5] == 1 and wins[game-4] == 1 and wins[game-3] == 1 and wins[game-2] == 1 and wins[game-1] == 1 and episode == 0:
          episode = game
        t += 1

        action = choose_action(state)

        state2, reward, done, info = env.step(action)

        if t == max_steps:
            done = True  

        learn(state, state2, reward, action, done)

        state = state2

        if done and reward == 1:
            wins.append(1)
            break
        if done:
            wins.append(0)
            break        


print("Количество побед в серии из 10 000 игр: ", np.sum(wins))
print("Пять побед подряд впервые было одержано в игре ", episode)

Ваша карта

SFFFHH
FFFFFF
FFFFFF
FFFFFF
FFFFFH
HFHHFG


100%|██████████| 10000/10000 [00:13<00:00, 748.10it/s]

Количество побед в серии из 10 000 игр:  9116
Пять побед подряд впервые было одержано в игре  32
